In [1]:
def generarPasos(u, x):
    res = 0
    if x < 0.5:
        res = u*x
    elif x >= 0.5:
        res = u*(1-x)
    return res

In [2]:
x = 0.1
for i in range(50):
    print(f"X{i+1}: {x:.4f}")
    x = generarPasos(1.5, x)

X1: 0.1000
X2: 0.1500
X3: 0.2250
X4: 0.3375
X5: 0.5063
X6: 0.7406
X7: 0.3891
X8: 0.5836
X9: 0.6246
X10: 0.5631
X11: 0.6554
X12: 0.5169
X13: 0.7246
X14: 0.4131
X15: 0.6197
X16: 0.5705
X17: 0.6443
X18: 0.5336
X19: 0.6996
X20: 0.4505
X21: 0.6758
X22: 0.4863
X23: 0.7295
X24: 0.4058
X25: 0.6087
X26: 0.5870
X27: 0.6195
X28: 0.5707
X29: 0.6439
X30: 0.5341
X31: 0.6989
X32: 0.4517
X33: 0.6775
X34: 0.4837
X35: 0.7256
X36: 0.4116
X37: 0.6175
X38: 0.5738
X39: 0.6393
X40: 0.5411
X41: 0.6884
X42: 0.4674
X43: 0.7011
X44: 0.4483
X45: 0.6724
X46: 0.4913
X47: 0.7370
X48: 0.3945
X49: 0.5917
X50: 0.6124


In [3]:
# ============================================
# INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# ============================================

# --- Import packages ---
try:
    import micropip
    await micropip.install(['ipywidgets', 'bqplot', 'plotly'])
except (ImportError, AttributeError, NameError):
    import sys
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ipywidgets', 'bqplot', 'plotly'])

# --- Test imports ---
try:
    import ipywidgets as widgets
    print("✓ ipywidgets importado correctamente")
except ImportError:
    print("✗ Error al importar ipywidgets")

try:
    import bqplot as bq
    print("✓ bqplot importado correctamente")
except ImportError:
    print("✗ Error al importar bqplot")

try:
    import plotly.graph_objects as go
    print("✓ plotly importado correctamente")
except ImportError:
    print("✗ Error al importar plotly")

# --- JavaScript patch for bqplot dragging ---
from IPython.display import display, Javascript
display(Javascript("""
// Patch for bqplot Scatter.enable_move in JupyterLite
(function() {
    if (typeof window.bqplotScatterPatchApplied !== 'undefined') return;
    
    function waitForBqplot() {
        if (typeof window.require === 'undefined') {
            setTimeout(waitForBqplot, 100);
            return;
        }
        
        require(['bqplot'], function(bqplot) {
            if (!bqplot || !bqplot.Scatter || !bqplot.Scatter.prototype) {
                console.warn('bqplot.Scatter no encontrado para parchear');
                return;
            }
            
            const originalInitialize = bqplot.Scatter.prototype.initialize;
            bqplot.Scatter.prototype.initialize = function(options) {
                const result = originalInitialize.apply(this, arguments);
                if (this.model.get('enable_move')) {
                    this._enableDrag();
                }
                return result;
            };
            
            if (!bqplot.Scatter.prototype._enableDrag) {
                bqplot.Scatter.prototype._enableDrag = function() {
                    if (!this.d3el || !this.d3el.selectAll) return;
                    
                    const that = this;
                    this.d3el.selectAll('.object_grp')
                        .style('cursor', 'move')
                        .call(d3.drag()
                            .on('drag', function(event, d) {
                                const scales = that.model.get('scales');
                                const xScale = scales.x;
                                const yScale = scales.y;
                                
                                if (xScale && yScale) {
                                    const newX = xScale.scale.invert(event.x);
                                    const newY = yScale.scale.invert(event.y);
                                    
                                    that.model.set('x', [newX]);
                                    that.model.set('y', [newY]);
                                    that.model.save_changes();
                                }
                            })
                        );
                };
            }
            
            window.bqplotScatterPatchApplied = true;
            console.log('✓ Parche de arrastre bqplot aplicado correctamente');
        }, function(err) {
            console.warn('No se pudo cargar bqplot para parchear:', err);
        });
    }
    
    if (document.readyState === 'loading') {
        document.addEventListener('DOMContentLoaded', waitForBqplot);
    } else {
        waitForBqplot();
    }
})();
"""))

print("\n🎉 ¡Todo listo! Ejecuta la siguiente celda para el widget interactivo.")



✓ ipywidgets importado correctamente
✓ bqplot importado correctamente
✓ plotly importado correctamente


<IPython.core.display.Javascript object>


🎉 ¡Todo listo! Ejecuta la siguiente celda para el widget interactivo.


In [4]:
from IPython.display import HTML
from ipywidgets import interact
from plotly.subplots import make_subplots
from plotly.io import to_html

In [5]:
def secuencia(a, u, num):
    x = a
    res = [[],[a]]
    for i in range(num):
        res[0].append(i)
    for i in range(num-1):
        x = generarPasos(u, x)
        res[1].append(x)
    return res
        

In [18]:
def mapa_logistico_clasico(r, x):
    """Calcula x_{n+1} usando el mapa clásico."""
    return r * x * (1 - x)

def generar_secuencia_clasica(r, x0, n_pasos):
    """Genera secuencia con el mapa clásico."""
    valores = [[],[x0]]
    for i in range(n_pasos):
        valores[0].append(i)
    for i in range(n_pasos - 1):
        valores[1].append(mapa_logistico_clasico(r, valores[1][i]))
    return valores
    
def cobweb_sequence(start, r, steps):
    xs, ys = [start], []
    x = start
    for i in range(steps):
        y = r * x * (1 - x)
        ys.append(y)
        x = y
        xs.append(x)
    return xs, ys

In [7]:
print(generar_secuencia_clasica(r=2, x0=0.5, n_pasos=20))

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]


In [24]:
mu = 2
start = 0.9
steps = 50
r = 2
x_range = [0, 100]
y_range = [0, 1]
y_dtick = 0.1
def update(mu=2.0, start=0.9, steps=50, r=2.0):
    xs, ys = secuencia(start, mu, steps)
    xl, yl = generar_secuencia_clasica(r, start, steps)
    fig = make_subplots(rows=4, cols=1, subplot_titles=[f'caos determinista y mapa logístico(mu={mu:.2f} r={r:.2f})', 
                                                        f'caos determinista(mu={mu:.2f})',
                                                        f'mapa logístico(r={r:.2f})',
                                                        f'profe pongame un 7 (r={r:.2f})'])
    det = go.Scatter(x=xs, y=ys, mode="lines+markers", name="modelo caos determinista")
    log = go.Scatter(x=xl, y=yl, mode="lines+markers", name="modelo logístico")
    fig.add_trace(det, 1, 1)
    fig.add_trace(log, 1, 1)
    fig.add_trace(det, 2, 1)
    fig.add_trace(log, 3, 1)
    #diagrama de telaraña
    diag_x = [0, 1]
    diag_y = [0, 1]
    map_x = [i/100 for i in range(101)]
    map_y = [r * xi * (1 - xi) for xi in map_x]  # map for cobweb
    
    fig.add_trace(go.Scatter(x=diag_x, y=diag_y, mode='lines', line=dict(color='black', dash='dash'), name='y=x'), 4, 1)
    fig.add_trace(go.Scatter(x=map_x, y=map_y, mode='lines', line=dict(color='green'), name='y=mu*x*(1-x)'), 4, 1)
    
    cobweb_xs, cobweb_ys = cobweb_sequence(start, r, steps)
    for i in range(steps):
        # Vertical line
        fig.add_trace(go.Scatter(x=[cobweb_xs[i], cobweb_xs[i]], y=[cobweb_xs[i], cobweb_ys[i]], 
                                 mode='lines', line=dict(color='blue', width=1)), 4, 1)
        # Horizontal line
        fig.add_trace(go.Scatter(x=[cobweb_xs[i], cobweb_ys[i]], y=[cobweb_ys[i], cobweb_ys[i]], 
                                 mode='lines', line=dict(color='red', width=1)), 4, 1)
    for i in range(1, 4):
        fig.update_xaxes(range=x_range, row=i, col=1)
        fig.update_yaxes(range=y_range, row=i, col=1, dtick=y_dtick)
    fig.update_layout(title="análisis del modelo de caos determinista y mapa logístico", width=900, height=1500)
    display(HTML(to_html(fig, include_plotlyjs="cdn", full_html=False)))

interact(update, mu=(0, 2, 0.1), start=(0, 1, 0.1), steps=(1, 200, 1), r=(0, 4, 0.1))

interactive(children=(FloatSlider(value=2.0, description='mu', max=2.0), FloatSlider(value=0.9, description='s…

<function __main__.update(mu=2.0, start=0.9, steps=50, r=2.0)>